In [77]:
import os
import json 
from openai import OpenAI
from dotenv import load_dotenv
from collections import defaultdict

load_dotenv()

True

In [9]:
# Send a trading request to OpenAI API
client = OpenAI(api_key=os.environ.get(os.environ.get("OPENAI_API_KEY"))) 
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": os.environ.get("ROLE")},
        {"role": "user", "content": os.environ.get("INSTRUCTIONS")}
    ],
    model="gpt-4o",
)

In [25]:
# Clean the artefacts to get a string
result = str(chat_completion.choices[0].message)
result = result.strip('ChatCompletionMessage(content=')
result = result.strip(", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)")

In [74]:
# Function to parse the strategies
from collections import defaultdict

def parse_strategies(raw_text):
    raw_text = raw_text.replace("\\n", "\n").strip()
    raw_text = raw_text.replace("\n", "").strip()
    strategy_dict = {}
    # Split strategies based on the '### Strategy' separator
    strategies = re.split(r'### Strategy (\d+)', raw_text)
    strategies = strategies[1:]  # Remove empty prefix
    for i in range(0, len(strategies), 2):
        strategy_number = f"Strategy {strategies[i]}"
        strategy_content = strategies[i + 1].strip()
        # Parse the content of each strategy
        sections = re.split(r'\*\*(.+?):\*\*', strategy_content)
        sections = sections[1:]  # Remove empty prefix
        strategy_data = defaultdict(str)
        for j in range(0, len(sections), 2):
            key = sections[j].strip()
            value = sections[j + 1].strip()
            strategy_data[key] = value
        strategy_dict[strategy_number] = dict(strategy_data)
    return strategy_dict

# Parse the raw text
strategies = parse_strategies(result)

In [79]:
d = parse_strategies(result)
d

{'Strategy 1': {'Composition': 'The typical 3-2-1 crack spread involves buying three crude oil futures contracts and simultaneously selling two gasoline futures and one distillate fuel oil futures (such as diesel) contracts, reflecting the typical yields from refining one barrel of crude oil.',
  'Trading Strategy': 'In this strategy, you would establish a position in the 3-2-1 crack spread and then hedge this position with a refiner stock such as Valero Energy Corporation (VLO). The goal is to profit from the differential between the movements of crack spreads and the stock price.',
  'Economic Rationale': 'Refiner stocks are sensitive to crack spreads since they represent the gross margin (before operating costs) that refiners earn for converting crude oil into refined products. Thus, refiner stocks usually react positively to widening crack spreads because it implies higher profitability margins. Conversely, narrowing spreads may lead to declines in refiner stock prices.',
  'Trade 

In [80]:
# Convert and write JSON object to file
with open("trading_strategies.json", "w") as outfile: 
    json.dump(d, outfile)